# Novel DDPM Inpainting — Quantitative Evaluation

This notebook evaluates `novel_inpaint.py` (cosine-decayed mask dilation) on **N images**
from the **MS COCO val2017** dataset, using **MNIST digit silhouettes** as masks (~10% of image).

Results are directly comparable to `Vanilla-Inpainting-Demo.ipynb` and `Improved-Inpainting-Demo.ipynb`
— all notebooks use the **same SEED and N_IMAGES**, so they sample identical COCO images and MNIST digits.

**Novel contribution:** at early (noisy) timesteps the mask is dilated so the model can anchor
generated content into surrounding context. The dilation radius decays to zero via a cosine
schedule, letting edges snap back to the true mask boundary for fine detail.

**Output layout:**
```
eval_results_novel/
  inpainted/            ← 0000.png … 0999.png
  checkpoint.json
  metric_distributions.png
  top10_best.png
  top10_worst.png
```
Originals and masks are reused from `eval_results/` (written by the vanilla notebook).

In [ ]:
import sys, os, warnings
import torch
import numpy as np
warnings.filterwarnings("ignore", message="IProgress not found")
from tqdm.auto import tqdm
from utils import (
    load_data, prepare_mnist_mask, prepare_coco_image, apply_mask_for_display,
    run_metrics,
    load_checkpoint, save_checkpoint,
    print_stats_table, plot_kde_single, show_top10,
)

# ---- Config ----
N_IMAGES       = 20
IMAGE_SIZE     = (512, 512)
SEED           = 42
STEPS          = 50
GUIDANCE_SCALE = 7.5
DATA_ROOT      = './data'
MAX_RADIUS     = 8
DECAY_STEPS    = 25

RESULTS_DIR    = './eval_results_novel'
VANILLA_DIR    = './eval_results'   # reuse originals/ and masks/ from vanilla run

os.makedirs(os.path.join(RESULTS_DIR, 'inpainted'), exist_ok=True)

torch.manual_seed(SEED)
np.random.seed(SEED)

print(f'Config: N_IMAGES={N_IMAGES}, SIZE={IMAGE_SIZE}, STEPS={STEPS}, GUIDANCE={GUIDANCE_SCALE}')
print(f'Novel params: MAX_RADIUS={MAX_RADIUS}, DECAY_STEPS={DECAY_STEPS}')
print(f'Output dir: {RESULTS_DIR}/inpainted')

In [ ]:
selected_ids, id_to_filename, captions, mnist, mnist_indices = load_data(
    DATA_ROOT, seed=SEED, n_images=N_IMAGES
)

coco_img_dir = os.path.join(DATA_ROOT, 'coco', 'val2017')
coco_samples = [
    (id_to_filename[img_id], captions[img_id][0])
    for img_id in selected_ids
]

In [ ]:
import matplotlib.pyplot as plt

sample_filename, sample_caption = coco_samples[0]
sample_img   = prepare_coco_image(os.path.join(coco_img_dir, sample_filename))
sample_digit = mnist[mnist_indices[0]][0]
sample_mask  = prepare_mnist_mask(sample_digit)
sample_masked = apply_mask_for_display(sample_img, sample_mask)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(sample_img);                          axes[0].set_title('Original (COCO)')
axes[1].imshow(sample_mask.squeeze(), cmap='gray');  axes[1].set_title('Mask (1=keep, 0=inpaint)')
axes[2].imshow(sample_masked);                       axes[2].set_title('Masked')
for ax in axes:
    ax.axis('off')
plt.suptitle(f'Caption: "{sample_caption}"', fontsize=10, y=1.01)
plt.tight_layout()
plt.show()

frac = (sample_mask == 0).float().mean().item() * 100
print(f'Inpainted region: {frac:.1f}% of image')

In [ ]:
sys.path.insert(0, os.path.abspath('.'))
from novel_inpaint import ddpm_inpaint_novel
from utils.cli import load_sd_pipeline

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using device: {device}')

print('Loading model...')
pipe = load_sd_pipeline(device)
pipe.set_progress_bar_config(disable=True)
print('Model loaded.')

In [ ]:
import json

INPAINTED_DIR = os.path.join(RESULTS_DIR, 'inpainted')
MASKS_DIR     = os.path.join(VANILLA_DIR, 'masks')      # reuse from vanilla
ORIGINALS_DIR = os.path.join(VANILLA_DIR, 'originals')  # reuse from vanilla

start_idx = load_checkpoint(RESULTS_DIR)

for i in tqdm(range(start_idx, N_IMAGES), initial=start_idx,
              total=N_IMAGES, desc='Novel inpainting'):
    filename, prompt = coco_samples[i]
    img_pil     = prepare_coco_image(os.path.join(coco_img_dir, filename))
    mask_tensor = prepare_mnist_mask(mnist[mnist_indices[i]][0])

    inpainted_pil = ddpm_inpaint_novel(
        pipe=pipe,
        image=img_pil,
        mask=mask_tensor,
        prompt=prompt,
        steps=STEPS,
        guidance_scale=GUIDANCE_SCALE,
        seed=SEED + i,
        max_radius=MAX_RADIUS,
        decay_steps=DECAY_STEPS,
    )

    inpainted_pil.save(os.path.join(INPAINTED_DIR, f'{i:04d}.png'))
    save_checkpoint(RESULTS_DIR, i + 1, prompt)

print(f'Done. Inpainted {N_IMAGES} images.')

In [ ]:
results = run_metrics(
    inpainted_dir=os.path.join(RESULTS_DIR, 'inpainted'),
    masks_dir=os.path.join(VANILLA_DIR, 'masks'),
    originals_dir=os.path.join(VANILLA_DIR, 'originals'),
    n_images=N_IMAGES,
    device=device,
)

print(f'Metrics computed for {len(results)} images.')

In [ ]:
print_stats_table(results, label='Novel DDPM Inpainting (Cosine-Decayed Mask Dilation)')

plot_kde_single(
    results,
    out_path=os.path.join(RESULTS_DIR, 'metric_distributions.png'),
    title='Novel DDPM Inpainting — Metric Distributions',
)

In [ ]:
show_top10(
    results,
    sort_by='lpips', ascending=True,
    out_path=os.path.join(RESULTS_DIR, 'top10_best.png'),
)

In [ ]:
show_top10(
    results,
    sort_by='lpips', ascending=False,
    out_path=os.path.join(RESULTS_DIR, 'top10_worst.png'),
)